In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
test.head()

# Removing stopwords from titles and finding matches

In [ ]:
same_group = train.iloc[np.where(train['label_group'] == 3627744656)]
same_group.head()

In [ ]:
print(same_group.title.value_counts())
print(len(np.unique(same_group.title)))

We can see how many of the titles in this group have similar words, but are still slightly different. Let's see if we can combine some of these by removing stopwords.

In [ ]:
lowercase = [title.lower() for title in same_group.title]
tokenized = [word_tokenize(title) for title in lowercase]

stopwords_punctuation = stopwords.words('english')
punctuation = ['&', '-', '(', ')', '/', '|']
stopwords_punctuation.extend(punctuation)

titles_no_stopwords = []
for title in tokenized:
    title_no_sw = [word for word in title if not word in stopwords_punctuation]
    titles_no_stopwords.append(title_no_sw)
    
print(titles_no_stopwords)

In [ ]:
same_group['title_no_sw'] = titles_no_stopwords
same_group.head()

In [ ]:
print(same_group.title_no_sw.value_counts())
print(len(np.unique(same_group.title_no_sw)))

It looks like we have reduced the number of unique values in the titles! Let's try expanding this to the entire dataset.

In [ ]:
lowercase = [title.lower() for title in train.title]
tokenized = [word_tokenize(title) for title in lowercase]

stopwords_punctuation = stopwords.words('english')
punctuation = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\t','\n',"'",",",'~' , '—']
stopwords_punctuation.extend(punctuation)

titles_no_stopwords = []
for title in tokenized:
    title_no_sw = [word for word in title if not word in stopwords_punctuation]
    title_string = ' '.join(title_no_sw)
    titles_no_stopwords.append(title_string)
    
train['title_no_sw'] = titles_no_stopwords
train.head()

In [ ]:
start_time = time.time()

ids = []
item_group = [] # tracks which group an item is in
groups = {} # tracks which items are in each group
for ind in train.index:
    
    indices_phash = train.loc[(train['image_phash'] == train['image_phash'][ind])].index 
    indices_title = train.loc[(train['title_no_sw'] == train['title_no_sw'][ind])].index 
    indices_image = train.loc[(train['image'] == train['image'][ind])].index
    indices = set(indices_phash).union(set(indices_title)).union(set(indices_image))
    
    # check if an item this matches already has a group
    match_ids = list(train.loc[indices]['posting_id'])
    if set(match_ids).intersection(ids):
        existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
        item_group.append(existing_group)
        groups[existing_group] = set(groups[existing_group]).union(match_ids)
    else:
        item_group.append(len(groups))
        groups[len(groups)] = match_ids
        
    ids.append(train['posting_id'][ind])
    
matches = [' '.join(list(groups[ind])) for ind in item_group]

print(time.time() - start_time)

Now lets apply this to the test set.

In [ ]:
lowercase = [title.lower() for title in test.title]
tokenized = [word_tokenize(title) for title in lowercase]

stopwords_punctuation = stopwords.words('english')
punctuation = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\t','\n',"'",",",'~' , '—']
stopwords_punctuation.extend(punctuation)

titles_no_stopwords = []
for title in tokenized:
    title_no_sw = [word for word in title if not word in stopwords_punctuation]
    title_string = ' '.join(title_no_sw)
    titles_no_stopwords.append(title_string)
    
test['title_no_sw'] = titles_no_stopwords
test.head()

In [ ]:
start_time = time.time()

ids = []
item_group = [] # tracks which group an item is in
groups = {} # tracks which items are in each group
for ind in test.index:
    
    indices_phash = test.loc[(test['image_phash'] == test['image_phash'][ind])].index 
    indices_title = test.loc[(test['title_no_sw'] == test['title_no_sw'][ind])].index 
    indices_image = test.loc[(test['image'] == test['image'][ind])].index
    indices = set(indices_phash).union(set(indices_title)).union(set(indices_image))
    
    # check if an item this matches already has a group
    match_ids = list(test.loc[indices]['posting_id'])
    if set(match_ids).intersection(ids):
        existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
        item_group.append(existing_group)
        groups[existing_group] = set(groups[existing_group]).union(match_ids)
    else:
        item_group.append(len(groups))
        groups[len(groups)] = match_ids
        
    ids.append(test['posting_id'][ind])
    
matches = [' '.join(list(groups[ind])) for ind in item_group]

print(time.time() - start_time)

In [ ]:
submission = pd.DataFrame({'posting_id': ids, 'matches': matches})
submission.to_csv('submission.csv', index = False)
submission

# Finding Similar Titles with RAPIDS and kNN

https://www.kaggle.com/ruchi798/shopee-eda-rapids-preprocessing

https://www.kaggle.com/cdeotte/rapids-cuml-tfidfvectorizer-and-knn#Find-Similar-Titles-with-RAPIDS!

RAPIDS is a suite of open source software liraries that allow you to run code on GPUs. cuDF is a Python GPU DataFrame library. CuPy is a NumPy-compatible array library. cuML is a suite of GPU-accelerated ML algorithms.

In [ ]:
import cuml, cudf, cupy
import nltk
import tensorflow as tf

import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
print('RAPIDS',cuml.__version__)

In [ ]:
# load data to use with RAPIDS
train_c = cudf.from_pandas(train)
train_c.head()

## Preprocessing: TfidVectorizer

https://medium.com/rapids-ai/natural-language-processing-text-preprocessing-and-vectorizing-at-rocking-speed-with-rapids-cuml-74b8d751812e

TfidfVectorizer performs fast text vectorizing on GPUs, which can be almost 5 times faster than sklearn's `TfidfVectorizer` on CPUs. The TF-IDF score is the product of the term frequency and the inverse document frequency. Term frequency is the number of times each term appears in a document divided by the total number of words in the document. Inverse document frequency is the log of the number of documents divided by the number of documents that contain the word. `TfidfVectorizer` returns a cuPy sparse matrix.

In [ ]:
model = TfidfVectorizer(stop_words='english', binary=True)
text_embeddings = model.fit_transform(train_c.title).toarray()
print('text embeddings shape is',text_embeddings.shape)

## Fit a kNN model to find similar titles

In [ ]:
model = NearestNeighbors(n_neighbors = 50)
model.fit(text_embeddings)
distances, indices_kNN = model.kneighbors(text_embeddings)

Let's look at the nearest neighbors of the first few rows of our dataset.

In [ ]:
for k in range(5):
   plt.figure(figsize=(20,3))
   plt.plot(np.arange(50),cupy.asnumpy(distances[k,]),'o-')
   plt.title('Text Distance From Train Row %i to Other Train Rows'%k,size=16)
   plt.ylabel('Distance to Train Row %i'%k,size=14)
   plt.xlabel('Index Sorted by Distance to Train Row %i'%k,size=14)
   plt.show()
    
   print(train_c.loc[cupy.asnumpy(indices_kNN[k,:10]),['title','label_group']])

For 4 out of these 5 rows, the neighbors with a distance less than 0.9 seem to be in the same label group. In train row 3, the 3 nearest neighbors all have titles **very** similar to the original, but are surprisingly in different groups. We can try using 0.9 as our threshold and see where that gets us.

# Assembling the Overall Model

In [ ]:
start_time = time.time()

train_ids = []
train_item_group = [] # tracks which group an item is in
train_groups = {} # tracks which items are in each group
for ind in train.index:
    
    indices_phash = np.where(train.image_phash == train['image_phash'][ind])[0]
    indices_title = indices_kNN[ind,][cupy.where(distances[ind,] <= 0.9)]
    indices_image = np.where(train['image'] == train['image'][ind])[0]
    indices = set(indices_phash).union(set(list(cupy.asnumpy(indices_title)))).union(set(indices_image))
    
    # check if an item this matches already has a group
    match_ids = list(train.loc[indices]['posting_id'])
    if set(match_ids).intersection(train_ids):
        existing_group = train_item_group[train_ids.index(list(set(match_ids).intersection(train_ids))[0])]
        train_item_group.append(existing_group)
        train_groups[existing_group] = set(train_groups[existing_group]).union(match_ids)
    else:
        train_item_group.append(len(train_groups))
        train_groups[len(train_groups)] = match_ids
        
    train_ids.append(train['posting_id'][ind])
    
train_matches = [' '.join(list(train_groups[ind])) for ind in train_item_group]

print(time.time() - start_time)

In [ ]:
# add column with ground truth
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)

train['oof'] = [list(train_groups[ind]) for ind in train_item_group]
train.head()

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV score =',train.f1.mean())

This is a big improvement in the training score! Now let's apply this to test data.

# Fitting to Test Data and Generating Submission

In [ ]:
# load data to use with RAPIDS
test_c = cudf.read_csv('../input/shopee-product-matching/test.csv')
test_c.head()
#train_2 = cudf.from_pandas(train)
#test_c = cudf.concat([train_2,train_2],axis=0,ignore_index=True)
#test_c.head()

In [ ]:
model = TfidfVectorizer(stop_words='english', binary=True)
text_embeddings = model.fit_transform(test_c.title).toarray()
print('text embeddings shape is',text_embeddings.shape)

In [ ]:
model = NearestNeighbors(n_neighbors = 3)
if np.shape(test)[0] > 3:
    model = NearestNeighbors(n_neighbors = 50)

model.fit(text_embeddings)
distances, indices_kNN = model.kneighbors(text_embeddings)

In [ ]:
start_time = time.time()

ids = []
item_group = [] # tracks which group an item is in
groups = {} # tracks which items are in each group
for ind in test.index:
    
    indices_phash = np.where(test.image_phash == test['image_phash'][ind])[0]
    indices_title = indices_kNN[ind,][cupy.where(distances[ind,] <= 0.9)]
    indices_image = np.where(test['image'] == test['image'][ind])[0]
    indices = set(indices_phash).union(set(list(cupy.asnumpy(indices_title)))).union(set(indices_image))
    
    # check if an item this matches already has a group
    match_ids = list(test.loc[indices]['posting_id'])
    if set(match_ids).intersection(ids):
        existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
        item_group.append(existing_group)
        groups[existing_group] = set(groups[existing_group]).union(match_ids)
    else:
        item_group.append(len(groups))
        groups[len(groups)] = match_ids
        
    ids.append(test['posting_id'][ind])
    
matches = [' '.join(list(groups[ind])) for ind in item_group]

print(time.time() - start_time)

In [ ]:
submission = pd.DataFrame({'posting_id': ids, 'matches': matches})
submission.to_csv('submission.csv', index = False)
submission

Unfortunately, finding the nearest neighbor distances and indices gives us a memory error when running on the test set. In later notebooks we use a chunking method to avoid memory errors.